In [1]:
import pandas
import pandas as pd
import numpy as np
import operator
import glob
from os.path import expanduser as ospath
from urlparse import urlparse
import os
import requests
from requests_toolbelt.utils import dump
import unicodedata
import facebook
from collections import Counter


dir_name = 'data/statuses'
json_pattern = os.path.join(dir_name,'*.csv')
file_list = glob.glob(json_pattern)

507

In [4]:
# separating active pages from deactivated ones
map_df = pandas.read_csv('data/fb_pages_mapping.csv')
active_page_files = []
active_ids = list(map_df['id'])
total_post_count = 0
for each_file in file_list:
    page_id = each_file.split('/')[2].split('_')[0]
    if int(page_id) in active_ids:
        active_page_files.append(each_file)
    

In [5]:
# total count of posts
for each_file in active_page_files:
    each_df = pandas.read_csv(each_file)
    total_post_count += len(each_df)

In [7]:
# find out how many status types are present in the data
links_type_dict = {}
for each_file in active_page_files:
    links_counter = {}
    file_df = pandas.read_csv(each_file)
    links_counter = Counter(file_df['status_type'])
    page_id = each_file.split('/')[2].split('_')[0]
    page_name = list(map_df[map_df['id'] == int(page_id)]['page_name'])[0]
    links_type_dict[page_name] = links_counter


In [9]:
# dictionary indicating status type and total post count in each category
status_type_dict = {}
keys = ['event','link','music','note','offer','photo','status','video',]
for key, val in links_type_dict.items():
    for each_key in keys:
        if each_key in val.keys():
            if status_type_dict.has_key(each_key) == False:
                status_type_dict[each_key] = val[each_key]
            else:
                status_type_dict[each_key] += val[each_key]
    

In [9]:
status_type_dict

{'event': 1511,
 'link': 3501965,
 'music': 88,
 'note': 50,
 'offer': 19,
 'photo': 458183,
 'status': 30621,
 'video': 154108}

In [10]:
# total percentage of links
total = status_type_dict['event'] + status_type_dict['link'] + status_type_dict['photo'] + status_type_dict['video']+ status_type_dict['status']
float(status_type_dict['link']*100)/total

84.4582079631718

In [11]:
import urlclean
import requests
from bs4 import BeautifulSoup
import unicodedata
import pandas
import pandas as pd
from urlparse import urlparse
import json
# list of url shorteners
url_shorteners = ['bit.ly','goo.gl','tinyurl.com','is.gd','cli.gs','pic.gd','tweetphoto','DwarfURL.com','ow.ly','yfrog.com','migre.me','ff.im','tiny.cc','url4.eu','tr.im','twit.ac','su.pr','twurl.nl','snipurl.com','BudURL.com','short.to','ping.fm','Digg.com','post.ly','Just.as','.tk','bkite.com','snipr.com','flic.kr','loopt.us','doiop.com','twitthis.com','htxt.it','AltURL.com','RedirX.com','DigBig.com','short.ie','u.mavrev.com','kl.am','wp.me','u.nu','rubyurl.com','om.ly','linkbee.com','Yep.it','posted.at','xrl.us','metamark.net','sn.im','hurl.ws','eepurl.com','idek.net','urlpire.com','chilp.it','moourl.com','snurl.com','xr.com','lin.cr','EasyURI.com','zz.gd','ur1.ca','URL.ie','adjix.com','twurl.cc','s7y.us','shrinkify','EasyURL.net','atu.ca','sp2.ro','Profile.to','ub0.cc','minurl.fr','cort.as','fire.to','2tu.us','twiturl.de','to.ly','BurnURL.com','nn.nf','clck.ru','notlong.com','thrdl.es','spedr.com','vl.am','miniurl.com','virl.com','PiURL.com','1url.com','gri.ms','tr.my','Sharein.com','urlzen.com','fon.gs','Shrinkify.com','ri.ms','b23.ru','Fly2.ws','xrl.in','Fhurl.com','wipi.es','korta.nu','shortna.me','fa.b','WapURL.co.uk','urlcut.com','6url.com','abbrr.com','SimURL.com','klck.me','x.se','2big.at','url.co.uk','ewerl.com','inreply.to','TightURL.com','a.gg','tinytw.it','zi.pe','riz.gd','hex.io','fwd4.me','bacn.me','shrt.st','ln-s.ru','tiny.pl','o-x.fr','StartURL.com','jijr.com','shorl.com','icanhaz.com','updating.me','kissa.be','hellotxt.com','pnt.me','nsfw.in','xurl.jp','yweb.com','urlkiss.com','QLNK.net','w3t.org','lt.tl','twirl.at','zipmyurl.com','urlot.com','a.nf','hurl.me','URLHawk.com','Tnij.org','4url.cc','firsturl.de','Hurl.it','sturly.com','shrinkster.com','ln-s.net','go2cut.com','liip.to','shw.me','XeeURL.com','liltext.com','lnk.gd','xzb.cc','linkbun.ch','href.in','urlbrief.com','2ya.com','safe.mn','shrunkin.com','bloat.me','krunchd.com','minilien.com','ShortLinks.co.uk','qicute.com','rb6.me','urlx.ie','pd.am','go2.me','tinyarro.ws','tinyvid.io','lurl.no','ru.ly','lru.jp','rickroll.it','togoto.us','ClickMeter.com','hugeurl.com','tinyuri.ca','shrten.com','shorturl.com','Quip-Art.com','urlao.com','a2a.me','tcrn.ch','goshrink.com','DecentURL.com','decenturl.com','zi.ma','1link.in','sharetabs.com','shoturl.us','fff.to','hover.com','lnk.in','jmp2.net','dy.fi','urlcover.com','2pl.us','tweetburner.com','u6e.de','xaddr.com','gl.am','dfl8.me','go.9nl.com','gurl.es','C-O.IN','TraceURL.com','liurl.cn','MyURL.in','urlenco.de','ne1.net','buk.me','rsmonkey.com','cuturl.com','turo.us','sqrl.it','iterasi.net','tiny123.com','EsyURL.com','urlx.org','IsCool.net','twitterpan.com','GoWat.ch','poprl.com','njx.me']
json_data = open('fb_status_link.json').read()
fb_data = json.loads(json_data)
inactive_ids = ['1205242106174446', '125577181211429','1349936971683802', '1460363040662703','1488768428099602', '156351959061001', '167870080364375','1744605802516147', '1758142167775100', '1858711167741099', '299402180407111', '324090427983707', '428978587288437', '503743826481257', '54701331336','690344821147429', '862847370480630', '89742953276', '938667992913151', '942564539125675']
active_fb_pages = []
short_url_count_dict = {}

In [12]:
for each_file in file_list:
    page_id = each_file.split("/")[2].split("_")[0]
    if page_id not in inactive_ids:
        active_fb_pages.append(each_file)

In [13]:
# creating a dictionary of short urls for each Fb page
import json
short_url_dict = {}
json_data = open('fb_status_link.json').read()
fb_data = json.loads(json_data)
for key, val in fb_data.items():
    short_url_dict[key] = {}
    for each_key, each_val in val['status_link'].items():
        if each_key in url_shorteners:
            short_url_dict[key][each_key] = each_val


In [14]:
len(short_url_dict)

487

In [15]:
# pasges that do not contain any short urls
non_bitly_pages = []
for key, val in short_url_dict.items():
    if val == {}:
        non_bitly_pages.append(key)
        short_url_dict.pop(key)
len(non_bitly_pages)  

179

In [16]:
map_df = pandas.read_csv('data/fb_pages_mapping.csv')
active_ids = list(map_df['id'])
active_names = list(map_df['page_name'])

long_url_page_ids = list(map_df[map_df['page_name'].isin(non_bitly_pages)]['id'])

/Users/shwetabhatt/anaconda/lib/python2.7/site-packages/pandas/core/algorithms.py:399: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  f = lambda x, y: htable.ismember_object(x, values)


In [17]:
# list of page ids of Fb pages containing short urls as statuses
short_url_page_ids = []
for each_id in list(map_df['id']):
    if each_id not in long_url_page_ids:
        short_url_page_ids.append(each_id)
len(short_url_page_ids)    

309

In [19]:
# Find the total count of short urls across each Fb page
short_url_count_dict = {}

for key, val in short_url_dict.items():
    total_short_urls = 0
    for each_key, each_val in val.items():
        total_short_urls += each_val
    short_url_count_dict[key] = total_short_urls
        

In [20]:
# Find overall short url count 
total_short_url_count = 0

for each_val in short_url_count_dict.values():
    total_short_url_count += each_val
total_short_url_count

250817

In [10]:
from collections import Counter
from urlparse import urlparse, urlsplit

In [22]:

from bs4 import BeautifulSoup
import unicodedata

def get_number_of_clicks(page):
    soup = BeautifulSoup(page.text, 'html.parser')
    main_div = soup.find(id='main')
    try:
        clicks_data = main_div.find_all(class_='info-wrapper--clicks-text')
        html = list(soup.children)[2]
        body = list(html.children)[3]
        script = list(body.children)[11].get_text()
        str_script = unicodedata.normalize('NFKD', script).encode('ascii','ignore')
        clicks = str_script.split("\n")[4].split(":")[1].split(",")[0]
        return clicks
    except:
        return None


In [23]:
import requests
def get_unshortened_url(link):
    session = requests.Session()  # so connections are recycled
    resp = session.head(url, allow_redirects=True)
    return resp.url

In [26]:
from urlparse import urlparse, urlsplit
def get_short_url_list(filename):
    short_urls = []
    page_df = pd.DataFrame()
    page_df =  pandas.read_csv(filename)
    for each_link in list(page_df[page_df['status_type']=='link']['status_link']):
        try:
            if urlparse.urlparse(each_link).netloc in url_shorteners:
                short_urls.append(each_link) 
        except:
            pass
    return short_urls 
        

In [25]:
short_url_page_ids = [str(x) for x in short_url_page_ids]

In [27]:
short_url_file_list = []
for each_file in file_list:
    page_id = each_file.split('/')[2].split('_')[0]
    if page_id in short_url_page_ids:
        short_url_file_list.append(each_file)


In [30]:
len(short_url_file_list)

309

In [31]:
import urllib
def get_headers(short_url):
    headers = {'method':'POST', 'path':'/proxy/v3/link/clicks',
'accept':'*/*',
'accept-language':'en-US,en;q=0.8',
'alexatoolbar-alx_ns_ph':'AlexaToolbar/alx-4.0.1',
'content-length':'110',
'content-type':'application/x-www-form-urlencoded; charset=UTF-8',
'cookie':'intercom-id-qma0svun=943ed489-02dd-445f-9c1c-04556f609c74; _bit=h7bfi3-2befb01299f4c3931f-00n; _cc=2; _xsrf=a4dac62b1d03413c93f92e76ad20f2b1; anon_u=cHN1X19mNzMwZjFkYi1lOTQ2LTQyYTUtOTMxMC00ZGQwMDk5YThkYzE=|1502842219|8566baf831d3131d7b28f6389a798d31e7b250f1; _ga=GA1.2.1089544842.1502034674; _gid=GA1.2.860537510.1502842117; user=MXxvXzRrYTlldHB2aGQ=|1502897110|47925a78b6054f7c02a5581e0cfe9747b6968ac6; intercom-session-qma0svun=Z0l2WHFPMlJNc3cwL2pGcThUTzg0UzdUL1BaUWM3U2lnZy9jTU00TzhDcUo5SnVDWDhyc2t1eGVKRGhvN3VrWS0td2VIRnp6cVNSUGtKSi9RUHdQVHZqUT09--49995d19fa7de6c226427428b7b6dac2d7b9f38d',
'origin':'https://bitly.com',
'referer': short_url + '+',
'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
'x-bitly-client':'bbt2',
'x-xsrftoken':'a4dac62b1d03413c93f92e76ad20f2b1'}
    
    params = {'link':short_url,
    'timezone':0,
    'unit':'month',
    'units':22,
    'rollup':'false'
             }
    return headers, params

def get_clicks(short_url):
    encoded_url = urllib.quote_plus(short_url)
    bitly_url = 'https://api-ssl.bitly.com/v3/link/clicks?access_token=64a338894dd91bf6131b163fa5dc6aef284a2c52&link=%s&unit=day&units=-1&format=json'
    req_url = requests.get(bitly_url%(encoded_url))
    try:
        data = req_url.json()
        if data['status_code'] == 200:
            return data
        if data['status_code'] == 404:
            return 'non_bitly'
        if data['status_code'] == 403:
            return 'forbidden'
        return 'other'
    except:
        return 'other'

In [32]:
def compress_and_save(resp, f_name, d_type='json'):
    if d_type == 'json':
        resp = json.dumps(resp)
        f_name = f_name + '.json'
    else:
        f_name = f_name + '.txt.gz'
#     resp = resp.encode('utf-8').encode('zlib_codec')
    with open( f_name, 'w') as f:
        f.write (resp)

In [34]:
# Find clicks count for each bitly URL

import time
from collections import Counter
import json

counter = 0
sleep_time = 1.0
for each_file in short_url_file_list:
    page_id = each_file.split('/')[2].split('_')[0]
    bitly_url_stats = {}
    non_bitly_urls = []
    short_urls = []
    short_urls = get_short_url_list(each_file)
    if len(set(short_urls)) != 0:
        print len(short_urls)
        for each_link in list(set(short_urls))[500:1000]:
            clicks = get_clicks(each_link)
            if clicks == 'non_bitly':
                pass           
            elif clicks == 'forbidden':
                print each_link
                sleep_time += 5.0 
            elif clicks == 'other':
                print each_link
            else:
                sleep_time = 0.5
                path = '<path to directory>'+ page_id +"/"
                if not os.path.exists(path):
                    os.makedirs(path)
                compress_and_save(clicks, path + each_link.split("/")[2]+"_"+each_link.split("/")[3])
            counter += 1
            if counter == 1000:
                sleep_time += 5.0
            time.sleep(sleep_time)  
        if non_bitly_urls != []:
            with open('non_bitly_short_urls/'+page_id+'_non_bitly.txt', 'w') as f:
                f.write(json.dumps(non_bitly_urls, f))
        print counter 

data/statuses/107705785934333_facebook_statuses.csv
107705785934333
193
0
data/statuses/108247159506719_facebook_statuses.csv
108247159506719
5949
500
data/statuses/108728919161573_facebook_statuses.csv
108728919161573
6
500
data/statuses/1089660871073245_facebook_statuses.csv
1089660871073245
4
500
data/statuses/112114435501106_facebook_statuses.csv
112114435501106
2
500
data/statuses/1125333984241125_facebook_statuses.csv
1125333984241125
8
500
data/statuses/112623813202_facebook_statuses.csv
112623813202
3885
1000
data/statuses/112723252096438_facebook_statuses.csv
112723252096438
35
1000
data/statuses/1129007383778921_facebook_statuses.csv
1129007383778921
12
1000
data/statuses/114345125931_facebook_statuses.csv
114345125931
18
1000


## non_bitly_urls

In [111]:
if url_shorteners != []:
    with open('url_shorteners.txt', 'w') as f:
        f.write(json.dumps(url_shorteners))